<a href="https://colab.research.google.com/github/MohsenJadidi/Erdos-Goes-Neural/blob/main/ModifiedCode/Clique_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install package

In [9]:
# !pip install torch_scatter==latest+cu101 torch_sparse==latest+cu101
# !pip install torch_geometric


ERROR: Could not find a version that satisfies the requirement torch_scatter==latest+cu101 (from versions: 0.3.0, 1.0.2, 1.0.3, 1.0.4, 1.1.0, 1.1.1, 1.1.2, 1.2.0, 1.3.0, 1.3.1, 1.3.2, 1.4.0, 2.0.2, 2.0.3, 2.0.4, 2.0.5)
ERROR: No matching distribution found for torch_scatter==latest+cu101


In [ ]:
!pip install torch_scatter
!pip install torch_sparse
!pip install torch_geometric


In [10]:
!apt-get install python3-dev graphviz libgraphviz-dev pkg-config
!pip install graphviz
!pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [4]:
!pip install visdom
!pip install GPUtil

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('/content/drive/My Drive/ISLR Course - Fall 2020/Project')


Mounted at /content/drive/


# Import

In [1]:
import os
import pickle
import torch
import torch.nn.functional as F
from torch.nn import Linear
#from kernel.datasets import get_dataset
import time
from torch import tensor
from torch.optim import Adam
from torch.optim import SGD
from torch_geometric.data import DataLoader, DenseDataLoader as DenseLoader
from math import ceil
from torch.nn import Linear
from torch.distributions import categorical
from torch.distributions import Bernoulli
import torch.nn
%matplotlib inline
from matplotlib import pyplot as plt
import pygraphviz as pgv
from torch_geometric.utils import convert as cnv
from torch_geometric.utils import sparse as sp
from torch_geometric.data import Data
import pygraphviz as pgv
from networkx.drawing.nx_agraph import graphviz_layout
import networkx as nx
from torch.utils.data.sampler import RandomSampler
from torch.nn.functional import gumbel_softmax
from torch.distributions import relaxed_categorical
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.inits import glorot, zeros
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree
from torch_geometric.nn import GINConv, GATConv
from torch.nn import Parameter
from torch.nn import Sequential as Seq, Linear, ReLU, LeakyReLU
from torch_geometric.nn import MessagePassing
from torch.nn import Linear, Sequential, ReLU, BatchNorm1d as BN
#from torch_geometric.utils import scatter_
from torch_geometric.data import Batch 
from torch_scatter import scatter_min, scatter_max, scatter_add, scatter_mean
from torch import autograd
from torch_geometric.utils import softmax, add_self_loops, remove_self_loops, segregate_self_loops, remove_isolated_nodes, contains_isolated_nodes, add_remaining_self_loops
from models import cut_MPNN
from modules_and_utils import derandomize_cut, GATAConv,get_diracs, derandomize_clique_final, derandomize_clique_final_speed, solve_gurobi_maxclique
import scipy
import scipy.io
import GPUtil
from networkx.algorithms.approximation import max_clique
from torch_geometric.data import DataListLoader
from random import shuffle
from networkx.algorithms.approximation import max_clique
from networkx.algorithms import graph_clique_number
from networkx.algorithms import find_cliques

ModuleNotFoundError: ignored

# Read Dataset

In [81]:
import numpy as np
# datasets = ["TWITTER_SNAP", "COLLAB", "IMDB-BINARY"]
# dataset_name = datasets[2]
# path_to_dataset ="%PATH"
# if dataset_name == "TWITTER_SNAP":
#     stored_dataset = open(path_to_dataset, 'rb')        
# elif dataset_name == "COLLAB":
#     stored_dataset = open(path_to_dataset, 'rb')
# elif dataset_name == "IMDB-BINARY":
#     stored_dataset = open(path_to_dataset, 'rb')

address = '/content/drive/My Drive/ISLR Course - Fall 2020/Project/IMDB_BINARY.p'
stored_dataset = open(address, 'rb')      

dataset = pickle.load(stored_dataset)
dataset_scale = 1
# total_samples = int(np.floor(len(dataset)*dataset_scale))
total_samples = int(np.floor(dataset.len()*dataset_scale))

dataset = dataset[:total_samples]

# num_trainpoints = int(np.floor(0.6*len(dataset)))
# num_valpoints = int(np.floor(num_trainpoints/3))
# num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)
# traindata= dataset[0:num_trainpoints]
# valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
# testdata = dataset[num_trainpoints + num_valpoints:]
# batch_size = 32
# train_loader = DataLoader(traindata, batch_size, shuffle=True)
# test_loader = DataLoader(testdata, batch_size, shuffle=False)
# val_loader =  DataLoader(valdata, batch_size, shuffle=False)

# #set up random seeds 
# torch.manual_seed(1)
# np.random.seed(2)   
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

AttributeError: ignored

In [22]:
#type(dataset)

print(color.get(999))
type(color.get(999))


Data(edge_index=[2, 142], y=[1])


torch_geometric.data.data.Data

In [10]:
print(color.data)
type(color.data)

Data(edge_index=[2, 193062], y=[1000])


torch_geometric.data.data.Data

In [40]:
color.slices

{'edge_index': tensor([     0,    146,    402,  ..., 192744, 192920, 193062]),
 'y': tensor([   0,    1,    2,  ...,  998,  999, 1000])}

## Test other dataset

In [ ]:
!pip install spektral

In [1]:
from spektral.datasets import TUDataset

dataset = TUDataset('PROTEINS')
total_samples = int(np.floor(len(dataset)*dataset_scale))


dataset = dataset[:total_samples]
num_trainpoints = int(np.floor(0.6*len(dataset)))
num_valpoints = int(np.floor(num_trainpoints/3))
num_testpoints = len(dataset) - (num_trainpoints + num_valpoints)
traindata= dataset[0:num_trainpoints]
valdata = dataset[num_trainpoints:num_trainpoints + num_valpoints]
testdata = dataset[num_trainpoints + num_valpoints:]
batch_size = 32
train_loader = DataLoader(traindata, batch_size, shuffle=True)
test_loader = DataLoader(testdata, batch_size, shuffle=False)
val_loader =  DataLoader(valdata, batch_size, shuffle=False)

#set up random seeds 
torch.manual_seed(1)
np.random.seed(2)   
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

ModuleNotFoundError: ignored

# Train

In [95]:
def predict(model, data_loader, recfield):
    model.train()
    avg_loss = 0
    avg_cliqdist = 0
    exp_cardinalities = torch.tensor(0)
    for data in data_loader:
        optimizer.zero_grad()
        data = data.to(device)
        data = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = recfield)
        data = data.to(device)
        retdict = model(data)
        avg_loss += retdict['loss'][0].item()/len(data_loader)
        avg_cliqdist += retdict["Currvol/Cliquevol"][0].item()/len(data_loader)
        exp_cardinalities = [retdict["Expected_cardinality_hist"][0]]
    return avg_loss, avg_cliqdist, exp_cardinalities

In [96]:
b_sizes = [32]
l_rates = [0.001]
depths = [4]
coefficients = [4.]
rand_seeds = [66]
widths = [64]
epochs = 100
net.train()
retdict = {}
edge_drop_p = 0.0
edge_dropout_decay = 0.90
penalty_coeff = 9.00
penalty_increase = -0.00
validation_timeout = 75


#save_path = "%PATH"

for batch_size, learning_rate, numlayers, penalty_coeff, r_seed, hidden_1 in product(b_sizes, l_rates, depths, coefficients, rand_seeds, widths):
   
    torch.manual_seed(r_seed)
    train_loader = DataLoader(traindata, batch_size, shuffle=True)
    test_loader = DataLoader(testdata, batch_size, shuffle=False)
    val_loader =  DataLoader(valdata, batch_size, shuffle=False)

    receptive_field= numlayers + 1
    val_losses = []
    cliq_dists = []

    hidden_2 = 1

    net =  clique_MPNN(dataset,numlayers, hidden_1, hidden_2 ,1, elasticity = None)
    net.to(device).reset_parameters()
    optimizer = Adam(net.parameters(), lr=learning_rate, weight_decay=0.00000)
    
    plotter = VisdomLinePlotter(env_name=envname)
    for epoch in range(epochs):
        totalretdict = {}
        count=0
        if epoch % 5 == 0:
            edge_drop_p = edge_drop_p*edge_dropout_decay
            print("Edge_dropout: ", edge_drop_p)

        if epoch % 10 == 0:
            penalty_coeff = penalty_coeff + 0.
            print("Penalty_coefficient: ", penalty_coeff)

        if epoch % 15 == 0:
            val_l, cliq_dist, val_cardinalities= predict(net, val_loader, receptive_field)
            val_losses += [val_l]
            cliq_dists += [cliq_dist]
            if epoch>30:
                if (val_losses[-1] > val_losses[-2]) and (val_losses[-1] > val_losses[-3]) and (epoch>validation_timeout):
                    print("Converged!")
                    if val_losses[-2] < val_losses[-3]:
                        print("Best model epoch: ", epoch-15)
                        #file_name = save_path+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch-15)+'_samples.pt'
                        #net.load_state_dict(torch.load(file_name), strict=False)
                    else:
                        print("Best model epoch: ", epoch-30)
                        #file_name = save_path+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch-30)+'_samples.pt'
                        #net.load_state_dict(torch.load(file_name), strict=False)
                    break
                if epoch%15==0:
                    #file_name = save_path+ str(dataset_name)+ '/' + str(net)+ '_'+ str(epoch)+'_samples.pt'
                    print("file_name: ", file_name)
                    #torch.save({file_name : net.state_dict()},  file_name)

        if epoch % lr_decay_step_size == 0:
            for param_group in optimizer.param_groups:
                        param_group['lr'] = lr_decay_factor * param_group['lr']
        net.train()
        for data in train_loader:
            count += 1 
            optimizer.zero_grad(), 
            data = data.to(device)
            data_prime = get_diracs(data, 1, sparse = True, effective_volume_range=0.15, receptive_field = receptive_field)

            data = data.to('cpu')
            data_prime = data_prime.to(device)
            retdict = net(data_prime, edge_drop_p, penalty_coeff)
            
            for key,val in retdict.items():
                if "sequence" in val[1]:
                    if key in totalretdict:
                        totalretdict[key][0] += val[0].item()
                    else:
                        totalretdict[key] = [val[0].item(),val[1]]

            if epoch > 2:
                    retdict["loss"][0].backward()

                    torch.nn.utils.clip_grad_norm_(net.parameters(),1)
                    optimizer.step()
                    del(retdict)

            del data_prime

NameError: ignored

# Evaluate

In [ ]:
test_data = testdata
test_loader = DataLoader(test_data, batch-size, shuffle=False)
net.to(device)
count = 1
net.eval()

gnn_nodes = []
gnn_edges = []
gnn_sets = {}
#number of random samples
max_samples = 1
gnn_times = []
num_samples = max_samples
t_start = time.time()


for data in test_loader:
    num_graphs = data.batch.max().item()+1
    bestset = {}
    bestedges = np.zeros((num_graphs))
    maxset = np.zeros((num_graphs))
    
    #generate samples
    total_samples = []
    for graph in range(num_graphs):
        curr_inds = (data.batch==graph)
        g_size = curr_inds.sum().item()
        if max_samples <= g_size: 
            samples = np.random.choice(curr_inds.sum().item(),max_samples, replace=False)
        else:
            samples = np.random.choice(curr_inds.sum().item(),max_samples, replace=True)

        total_samples +=[samples]

    data = data.to(device)
    t_0 = time.time()
    for k in range(num_samples):
        t_datanet_0 = time.time()
        data_prime = get_diracs(data.to(device), 1, sparse = True, effective_volume_range=0.15, receptive_field = 7)
        print("Number of nodes: ", data_prime.x.shape[0])
        initial_values = data_prime.x.detach()
        data_prime.x = torch.zeros_like(data_prime.x)
        g_offset = 0
        for graph in range(num_graphs):
            curr_inds = (data_prime.batch==graph)
            g_size = curr_inds.sum().item()
            graph_x = data_prime.x[curr_inds]
            data_prime.x[total_samples[graph][k] + g_offset]=1.
            g_offset += g_size
       
        #forward_pass  
        ret_dict = net(data_prime)
        
        #derandomize
        t_datanet_1 = time.time() - t_datanet_0
        print("data prep and fp: ", t_datanet_1)
        t_derand_0 = time.time()
        _, set_edges, set_cardinality = derandomize_clique_final_speed(data_prime,(ret_dict["output"][0]), weight_factor =0.,draw=False, beam = 1)
        t_derand_1 = time.time() - t_derand_0
        print("Derandomization time: ", t_derand_1)
        for j in range(num_graphs):
            indices = (data.batch == j)
            if (set_cardinality[j]>maxset[j]):
                    maxset[j] = set_cardinality[j].item()
                    bestset[str(j)] = sets[indices].cpu()
                    bestedges[j] = set_edges[j].item()

    t_1 = time.time()-t_0
    print("Current batch: ", count)
    print("Time so far: ", time.time()-t_0)
    gnn_sets[str(count)] = bestset
    
    gnn_nodes += [maxset]
    gnn_edges += [bestedges]
    gnn_times += [t_1]




    count += 1
t_1 = time.time()
total_time = t_1 - t_start
print("Average time per graph: ", total_time/(len(test_data)))